In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'pandas'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pandas'



In [2]:
df = sqlContext.read.parquet("hdfs:///NLP-parquet/*.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- stemmed: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stars: double (nullable = true)
 |-- features: vector (nullable = true)

In [4]:
(train,test) = df.randomSplit([0.75,0.25], seed = 42)
(training,val) = train.randomSplit([0.75,0.25], seed = 42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
training.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- stemmed: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stars: double (nullable = true)
 |-- features: vector (nullable = true)

In [12]:
x_train = training.drop("stars","stemmed").toPandas()
x_val = val.drop("stars","stemmed").toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Pandas >= 0.19.2 must be installed; however, it was not found.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2086, in toPandas
    require_minimum_pandas_version()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 129, in require_minimum_pandas_version
    "it was not found." % minimum_pandas_version)
ImportError: Pandas >= 0.19.2 must be installed; however, it was not found.



In [8]:
x_train.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- features: vector (nullable = true)

In [9]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, ffnn_dim, n_heads):
        super(TransformerBlock, self).__init__()
        self.attention = layers.MultiHeadAttention(n_heads,key_dim=embed_dim)
        self.ffnn = keras.Sequential(
            [layers.Dense(ffnn_dim, activation = "relu"),
             layers.Dense(128, activation = "relu"),
             layers.Dense(embed_dim)]
        )
        
        self.norm1 = layers.LayerNormalization(epsilon = 1e-6)
        self.norm2 = layers.LayerNormalization(epsilon = 1e-6)
        self.drop1 = layers.Dropout(0.1)
        self.drop2 = layers.Dropout(0.1)
        
    def call(self, inputs, training):
        attention_out = self.attention(inputs, inputs)
        attention_out = self.drop1(attention_out, training=training)
        out1 = self.norm1(inputs + attention_out)
        ffnn_out = self.ffnn(out1)
        ffnn_out = self.drop2(ffnn_out, training=training)
        return self.norm2(out1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
class TokenAndPosEmbed(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPosEmbed, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim = embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
vocab_size = 65536
maxlen = 200

# test train split

# pad_sequence adds padding to each array suppose [1],[1,2],[1,2,3] are three
# arrays then pad_sequence will make it such that the output is [0,0,1],
# [0,1,2],[1,2,3]. Assuming array 3 is maxlen, i.e maxlen = 2.

x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_train = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
`sequences` must be iterable.
Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/tensorflow/python/keras/preprocessing/sequence.py", line 158, in pad_sequences
    padding=padding, truncating=truncating, value=value)
  File "/usr/local/lib64/python3.7/site-packages/keras_preprocessing/sequence.py", line 56, in pad_sequences
    raise ValueError('`sequences` must be iterable.')
ValueError: `sequences` must be iterable.



## Classifier call

In [5]:
embed_dim = 64 #embed size 
ffnn_dim = 64 #Neural network input dimension. Same as embed_dim
num_heads = 4

input_strm = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPosEmbed(maxlen, vocab_size, embed_dim)
nn = embedding_layer(input_strm)
transformer = TransformerBlock(embed_dim = embed_dim,ffnn_dim = ffnn_dim,n_heads = num_heads)(nn)
nn = layers.GlobalAveragePooling1D()(nn)
nn = layers.Dropout(0.1)(nn)
nn = layers.Dense(128, activation='relu')(nn)
nn = layers.Dropout(0.1)(nn)
outputs = layers.Dense(2, activation = "softmax")(nn)

cybertron = keras.Model(inputs = input_strm, outputs = outputs)

## Train and evaluate

In [7]:
cybertron.compile(optimizer = "adam", loss="categorical_crossentropy",metrics="accuracy")

history = cybertron.fit(
    x_train, y_train, batch_size = 32, epochs = 5, validation_data=(x_val, y_val)
)

NameError: name 'x_train' is not defined